<b>Sample dataset:</b> The dataset is specially created to understand decision tree. Here each row is a sample data and the last column is the label. The first two columns are features. 

<b>Suggestion:</b> Feel free to modify it by adding more features & examples.

<b>Interesting note:</b> In the current dataset 2nd and 5th examples have the same features, but different labels. This will help you to understand the concept better.


In [34]:
training_data = [['Green', 3, 'Mango'],['Yellow', 3, 'Mango'],['Red', 1, 'Grape'],['Red', 1, 'Grape'],['Yellow', 3, 'Lemon']]
# Column labels.
# These are used only to print the tree.
header = ["color", "diameter", "label"]

In [37]:
def unique_value(rows, col):
    #Find unique values for a column in a dataset.
    return set([row[col] for row in rows])

Testing the function <b>unique_value</b> to find the unique values from a particular column

In [62]:
a = unique_value(training_data, 0)
b = unique_value(training_data, 1)
c = unique_value(training_data, 2)
print( a,"\n",b ,"\n",c)

{'Green', 'Yellow', 'Red'} 
 {1, 3} 
 {'Mango', 'Grape', 'Lemon'}


In [90]:
def count_class(rows):
    #Counts the number of each type of example in a dataset.
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our sample dataset, we have label as the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

Testing the function <b>count_class</b> to find the count of each fruit(class)

In [91]:
count_class(training_data)

{'Mango': 2, 'Grape': 2, 'Lemon': 1}

In [92]:
def is_numeric(value):
    #Test if a value is numeric.
    return isinstance(value, int) or isinstance(value, float)

Testing the function <b>is_numeric</b> to test wheather a value is numeric or not

In [93]:
a = is_numeric(7)
b = is_numeric('Red')
print(a,b)

True False


In [94]:
class Question:# A Question is used to partition a dataset.

#Question will take two parameters: column number(e.g., 0 for Color) & column value(e.g., Green).

    #This __init__ function will allow passing the parameters to the class 'Question'
    def __init__(self, column, value):
        self.column = column
        self.value = value

    #The 'match' method is used to compare the feature value in an example to the feature value stored in the question.
    def match(self, example):
        # Compare the feature value in an example to the feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (header[self.column], condition, str(self.value))

Testing the class <b>Question</b>

In [95]:
q = Question(0, 'Green')
example1 = training_data[0]
example2 = training_data[1]
print(q.match(example1))
print(q.match(example2))

True
False


In [96]:
#This function is used to partition the dataset on the basis of the question.
# If the row in the dataset matches the question, add it to 'true rows' else 'false rows'
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

Testing the function: <b>partition</b> and dividing the dataset on the basis of some Questions

In [97]:
true_rows, false_rows = partition(training_data, Question(0, 'Red'))
print("When color is red: \n", true_rows)
print(false_rows,"\n")

true_rows, false_rows = partition(training_data, Question(0, 'Yellow'))
print("When color is yellow: \n", true_rows)
print(false_rows,"\n")

true_rows, false_rows = partition(training_data, Question(1, 3))
print("When diameter is 3: \n", true_rows)
print(false_rows,"\n")

When color is red: 
 [['Red', 1, 'Grape'], ['Red', 1, 'Grape']]
[['Green', 3, 'Mango'], ['Yellow', 3, 'Mango'], ['Yellow', 3, 'Lemon']] 

When color is yellow: 
 [['Yellow', 3, 'Mango'], ['Yellow', 3, 'Lemon']]
[['Green', 3, 'Mango'], ['Red', 1, 'Grape'], ['Red', 1, 'Grape']] 

When diameter is 3: 
 [['Green', 3, 'Mango'], ['Yellow', 3, 'Mango'], ['Yellow', 3, 'Lemon']]
[['Red', 1, 'Grape'], ['Red', 1, 'Grape']] 



In [104]:
def gini(rows):
    counts = count_class(rows)
    impurity = 1
    for label in counts:
        prob_of_label = counts[label] / float(len(rows))
        impurity -= prob_of_label**2
    return impurity

Testing the <b>gini</b> function to calculate the gini impurity

In [102]:
no_mixing = [['Mango'],['Mango']]
print("Gini Index - No mixing: ",gini(no_mixing))


some_mixing = [['Mango'],['Orange']]
print("Gini Index - Some mixing: ", gini(some_mixing))


lots_of_mixing = [['Banana'],['Orange'],['Grape'],['Apple'],['Mango']]
print("Gini Index - Lots of mixing: ", gini(lots_of_mixing))

Gini Index - No mixing:  0.0
Gini Index - Some mixing:  0.5
Gini Index - Lots of mixing:  0.7999999999999998


In [103]:
# Uncertainty of the starting node -  Weighted impurity of two child nodes.
def info_gain(left, right, current_uncertainty):
   
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

<b>DryRun:</b> Testing the info_gain function to see how it works.

In [109]:
# Demo: Calculate the uncertainy of our training data.
current_uncertainty = gini(training_data)
print("How much information do we gain by partioning on 'Green'?")
true_rows, false_rows = partition(training_data, Question(0, 'Green'))
print(info_gain(true_rows, false_rows, current_uncertainty))

print(" \nWhat about if we partioned on 'Red' instead?")
true_rows, false_rows = partition(training_data, Question(0,'Red'))
print(info_gain(true_rows, false_rows, current_uncertainty))

print("\n It looks like we learned more using 'Red' (0.37), than 'Green' (0.14)")
print("Why? Look at the different splits that result, and see which one looks more 'unmixed' to you")

true_rows, false_rows = partition(training_data, Question(0,'Red'))

print("\n Here, the true_rows contain only 'Grapes'")
print(true_rows)
print("\n And the false rows contain two types of fruit. Not too bad")
print(false_rows)

print("\n On the other hand, partitioning by Green doesn't help so much")
true_rows, false_rows = partition(training_data, Question(0,'Green'))

print("\n We've isolated one apple in the true rows.")
print(true_rows)

print("\n But, the false-rows are badly mixed up")
print(false_rows)

How much information do we gain by partioning on 'Green'?
0.1399999999999999
 
What about if we partioned on 'Red' instead?
0.37333333333333324

 It looks like we learned more using 'Red' (0.37), than 'Green' (0.14)
Why? Look at the different splits that result, and see which one looks more 'unmixed' to you

 Here, the true_rows contain only 'Grapes'
[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]

 And the false rows contain two types of fruit. Not too bad
[['Green', 3, 'Mango'], ['Yellow', 3, 'Mango'], ['Yellow', 3, 'Lemon']]

 On the other hand, partitioning by Green doesn't help so much

 We've isolated one apple in the true rows.
[['Green', 3, 'Mango']]

 But, the false-rows are badly mixed up
[['Yellow', 3, 'Mango'], ['Red', 1, 'Grape'], ['Red', 1, 'Grape'], ['Yellow', 3, 'Lemon']]


In [123]:
def find_best_split(rows):
    #Find the best question to ask by iterating over every feature / value and calculating the information gain.
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            if gain >=best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [124]:
# Demo: Find the best question to ask first from our sample dataset.
best_gain, best_question = find_best_split(training_data)
print("best_gain: ",best_gain,"\n","best_question: ",best_question)

best_gain:  0.37333333333333324 
 best_question:  Is diameter >= 3?


In [120]:
def find_best_split(rows):
    #Find the best question to ask by iterating over every feature / value and calculating the information gain.
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # You actually can use '>' instead of '>=' here 
            
            if gain >best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [121]:
# Demo: Find the best question to ask first from our sample dataset.
best_gain, best_question = find_best_split(training_data)
print("best_gain: ",best_gain,"\n","best_question: ",best_question)
# FYI: 'is color == Red' is just as good.

best_gain:  0.37333333333333324 
 best_question:  Is color == Red?


In [126]:
class Leaf:
    """A Leaf node classifies data. 
    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, rows):
        self.predictions = count_class(rows)


class Decision_Node:
    """A Decision Node asks a question.
    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,question,true_branch,false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch


def build_tree(rows):
     
    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain, and return the question that produces the highest gain.
    gain, question = find_best_split(rows)

    # Base case: no further info gain
    # Since we can ask no further questions,we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    # Return a Question node.
    # This records the best feature / value to ask at this point, as well as the branches to follow depending on the answer.
    return Decision_Node(question, true_branch, false_branch)


def print_tree(node, spacing=""):
   
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")


def classify(row, node):
    
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node, to the example we're refering to.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [136]:
# Demo: The tree predicts the 1st row of our training data is a mango with confidence 1.
my_tree = build_tree(training_data)

for i in range(len(training_data)):
    print(classify(training_data[i], my_tree),"\n")

{'Mango': 1} 

{'Mango': 1, 'Lemon': 1} 

{'Grape': 2} 

{'Grape': 2} 

{'Mango': 1, 'Lemon': 1} 



In [138]:
def print_leaf(counts):
    # A nicer way to print the predictions at a leaf.
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [146]:
# Demo: Printing that a bit nicer
print_leaf(classify(training_data[0], my_tree))
for i in range(len(training_data)):
    print(print_leaf(classify(training_data[i], my_tree)))

{'Mango': '100%'}
{'Mango': '50%', 'Lemon': '50%'}
{'Grape': '100%'}
{'Grape': '100%'}
{'Mango': '50%', 'Lemon': '50%'}


In [160]:
my_tree = build_tree(training_data)

print_tree(my_tree)

# Evaluate
testing_data = [
        ['Green', 3, 'Mango'],
        ['Yellow', 3, 'Mango'],
        ['Red', 2, 'Grape'],
        ['Red', 1, 'Grape'],
        ['Yellow', 3, 'Lemon'],
        ]

for row in testing_data:
        print ("Actual: %s. Predicted: %s" % (row[-1], print_leaf(classify(row, my_tree))))

# What to do next
# - add support for missing (or unseen) attributes
# - prune the tree to prevent overfitting
# - add support for regression


Is diameter >= 3?
--> True:
  Is color == Yellow?
  --> True:
    Predict {'Mango': 1, 'Lemon': 1}
  --> False:
    Predict {'Mango': 1}
--> False:
  Predict {'Grape': 2}
Actual: Mango. Predicted: {'Mango': '100%'}
Actual: Mango. Predicted: {'Mango': '50%', 'Lemon': '50%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Lemon. Predicted: {'Mango': '50%', 'Lemon': '50%'}


In [161]:
my_tree = build_tree(training_data)

print_tree(my_tree)

# Evaluate
testing_data = [
        ['Green', 3, 'Mango'],
        ['Yellow', 3, 'Mango'],
        ['Red', 2, 'Grape'],
        ['Red', 1, 'Grape'],
        ['Yellow', 3, 'Lemon']]
        

for row in testing_data:
        print ("Actual: %s. Predicted: %s" % (row[-1], print_leaf(classify(row, my_tree))))

# What to do next
# - add support for missing (or unseen) attributes
# - prune the tree to prevent overfitting
# - add support for regression


Is diameter >= 3?
--> True:
  Is color == Yellow?
  --> True:
    Predict {'Mango': 1, 'Lemon': 1}
  --> False:
    Predict {'Mango': 1}
--> False:
  Predict {'Grape': 2}
Actual: Mango. Predicted: {'Mango': '100%'}
Actual: Mango. Predicted: {'Mango': '50%', 'Lemon': '50%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Lemon. Predicted: {'Mango': '50%', 'Lemon': '50%'}
